# Use pre-trained VGG16 model

- [code example from fchollet](https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069)
- [related keras.io blog post](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
- [pixel-wise classification in keras](https://github.com/fchollet/keras/issues/1169)

The top layers of VGG16 are Flatten, Dense (fully-connected 1), Dense (fully-connected 2), Dense (prediction).

## To-do

1. ~~Use a generator when fitting and predicting.~~
2. Fix bug where batch size has to divide evenly into number of samples.
    - Is this in fact a bug? Or expected behavior.
3. Use open-cv instead of scipy to resize images. Open-cv is faster.
    - Original data is int64. Open-cv doesn't like that, but it likes float64. Normalize to range [0, 1], which will give a float (probably float64).

In [ ]:
import cv2
import keras
from keras.utils.io_utils import HDF5Matrix
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.models import Model, Sequential
from keras import optimizers
from keras.preprocessing.image import Iterator
import numpy as np
import tensorflow as tf

print("Keras", keras.__version__)
print("TensorFlow", tf.__version__)
print("NumPy", np.__version__)
print("OpenCV", cv2.__version__)

In [ ]:
params = {'data_path': '/data/metasearch-anatomical-brainmask-slices.h5',
          'slice_shape': (224, 224),
          'nb_train_start': 0,
          'nb_train_samples': 1000,
          'nb_test_start': 100000,
          'nb_test_samples': 1000,
          'epochs': 1,
          'batch_size': 25,
          'train_features_path': '/data/bottleneck_features_train.npy',
          'test_features_path': '/data/bottleneck_features_test.npy',
          'top_model_weights_path': '/data/top-model-weight.h5',
         }

In [ ]:
#--------------------------
# Data processing functions
#--------------------------

def _normalize(arr):
    """Return array normalized to range [0, 1]."""
    return (arr - arr.min()) / (arr.max() - arr.min())


def _normalize_and_resize_2d(arr2d, new_shape):
    """Return 2D array normalized to range [0, 1] and resized to `new_shape`."""
    return cv2.resize(_normalize(arr2d), new_shape, interpolation=cv2.INTER_NEAREST)
    
    # # Scipy resizing changes data range when using float64. OpenCV does not.
    # # OpenCV does not want to resize an image with int64 (why?), but this does
    # # concern us because normalizing returns float64.
    # # See https://github.com/scipy/scipy/issues/4458#issuecomment-269067103.
    #
    # return scipy.misc.imresize(arr2d, new_shape, interp='nearest')


def _process_3d(arr3d, new_shape):
    """Return 3D array that is normalized by slice to range [0, 1] and resized to
    `new_shape` on dims 1 and 2.
    """
    n_slices = arr3d.shape[0]
    processed_arr = np.zeros((n_slices, *new_shape))
    for idx in range(n_slices):
        processed_arr[idx, :, :] = _normalize_and_resize_2d(arr3d[idx, :, :],
                                                            new_shape)
    return processed_arr


def _process_2d(arr2d, new_shape):
    """Return 2D array that is normalized to range [0, 1] and resized to `new_shape`.
    """
    return _normalize_and_resize_2d(arr2d, new_shape)


def _process_shared(arr, new_shape):
    """Return processed batch of data (processing shared by X and y data).
    Array can be 2D or 3D.
    """
    try:
        return _process_3d(arr, new_shape)
    except IndexError:
        return _process_2d(arr, new_shape)


def _add_color_channels(arr):
    """Return array with values copied to mimic RGB color channels. Color channels
    are in the *last* axis.
    """
    arr.resize((*arr.shape, 1))
    return np.tile(arr, 3)


def process_X_data(arr, new_shape=params['slice_shape']):
    """Return processed batch of data. Resizes slices to `new_shape`
    and copies values to mimic RGB color channels.
    """
    arr = _process_shared(arr, new_shape=new_shape)
    return _add_color_channels(arr)


def process_y_data(arr, new_shape=params['slice_shape']):
    """Return processed batch of data. Resizes slices to `new_shape`."""
    return _process_shared(arr, new_shape=new_shape)


class SliceIterator(keras.preprocessing.image.Iterator):
    """Iterator yielding batches of data from 3D array of slices.
    
    Based on keras's NumpyArrayIterator:
    https://github.com/fchollet/keras/blob/master/keras/preprocessing/image.py#L735-L823
    """
    
    def __init__(self, X, y=None, batch_size=32, shuffle=False, seed=None):
        if y is not None and len(X) != len(y):
            raise ValueError('X and y should have the same length.')

        self.X = X
        self.y = y

        super(SliceIterator, self).__init__(X.shape[0], batch_size, shuffle, seed)


    def next(self):
        """For python 2.x (???)

        Returns
        -------
        The next batch. If `y` is not None, return tuple of X and y batches. Otherwise,
        return X batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)

        # # The code below is faster
        # upper_index = current_index + current_batch_size
        # self.X[current_index:upper_index], self.y[current_index:upper_index]

        if self.y is None:
            return self.X[index_array]

        return self.X[index_array], self.y[index_array]

In [ ]:
X_train = HDF5Matrix(params['data_path'], '/anatomical/axial', 
                     start=params['nb_train_start'], 
                     end=params['nb_train_start']+params['nb_train_samples'],
                     normalizer=process_X_data)

y_train = HDF5Matrix(params['data_path'], '/brainmask/axial',
                     start=params['nb_train_start'],
                     end=params['nb_train_start']+params['nb_train_samples'],
                     normalizer=process_y_data)

X_test = HDF5Matrix(params['data_path'], '/anatomical/axial',
                    start=params['nb_test_start'],
                    end=params['nb_test_start']+params['nb_test_samples'],
                    normalizer=process_X_data)

y_test = HDF5Matrix(params['data_path'], '/brainmask/axial',
                    start=params['nb_test_start'],
                    end=params['nb_test_start']+params['nb_test_samples'],
                    normalizer=process_y_data)

In [ ]:
print("Time taken to load and preprocess a single slice and a batch.")
print("batch size:", params['batch_size'], "slices")
%timeit X_train[0]
%timeit X_train[:params['batch_size']]

## Save bottleneck features

In [ ]:
def save_bottleneck_features():
    model = keras.applications.VGG16(include_top=False, weights='imagenet',
                                     input_shape=params['slice_shape']+(3,))

    # Instantiate generators of batches.
    generator_train_X = SliceIterator(X_train, batch_size=params['batch_size'])
    generator_test_X = SliceIterator(X_test, batch_size=params['batch_size'])

    bottleneck_features_train = model.predict_generator(
        generator_train_X, params['nb_train_samples'] // params['batch_size'],
        verbose=1)

    np.save(params['train_features_path'], bottleneck_features_train)


    bottleneck_features_validation = model.predict_generator(
        generator_test_X, params['nb_train_samples'] // params['batch_size'],
        verbose=1)

    np.save(params['test_features_path'], bottleneck_features_validation)

## Train top model

In [ ]:
def train_top_model():
    X_train_features = np.load(params['train_features_path'])
    X_test_features = np.load(params['test_features_path'])
    print("Train shape:", X_train_features.shape)
    print("Test shape:", X_test_features.shape)

    train_generator = SliceIterator(X_train_features, y_train, params['batch_size'])
    test_generator = SliceIterator(X_test_features, y_test, params['batch_size'])

    # https://github.com/fchollet/keras/issues/1169#issuecomment-162260972

    model = Sequential()
    model.add(Flatten(input_shape=X_train_features.shape[1:]))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Classification
    np.multiply(*params['slice_shape'])
    
    model.add(Dense(np.multiply(*params['slice_shape']), activation='sigmoid'))
    model.add(Reshape(params['slice_shape']))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy',
                  metrics=['accuracy'])

    steps = params['nb_train_samples'] // params['batch_size']

    model.fit_generator(train_generator, steps_per_epoch=steps, epochs=3,
                        validation_data=test_generator, validation_steps=steps,
                        verbose=1)

    model.save_weights(params['top_model_weights_path'])

In [ ]:
save_bottleneck_features()

In [ ]:
train_top_model()

## Train the joined model

In [ ]:
def join_models():
    # https://github.com/fchollet/keras/issues/4040#issuecomment-253309615
    
    initial_model = keras.applications.VGG16(include_top=False, weights='imagenet',
                                             input_shape=params['slice_shape']+(3,))

    x = Flatten()(initial_model.output)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    # Classification
    np.multiply(*params['slice_shape'])
    x = Dense(np.multiply(*params['slice_shape']), activation='sigmoid')(x)
    preds = Reshape(params['slice_shape'])(x)
    
    # Combine models.
    model = Model(initial_model.input, preds)

    # Freeze the convolutional part of the model.
    for layer in model.layers[:19]:  
        layer.trainable = False

    model.load_weights(params['top_model_weights_path'], by_name=True)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])

    return model

model = join_models()

In [ ]:
generator_train = SliceIterator(X_train, y_train, batch_size=params['batch_size'])
generator_test = SliceIterator(X_test, y_test, batch_size=params['batch_size'])

In [ ]:
model.fit_generator(
    generator_train,
    steps_per_epoch=params['nb_train_samples'] // params['batch_size'],
    epochs=params['epochs'],
    validation_data=generator_test,
    validation_steps=params['nb_test_samples'] // params['batch_size'],
    verbose=1)

In [ ]:
model.save("/data/vgg16-combined-model-20170630-first-try.h5")

In [ ]:
out = model.predict(X_test[49:50])

In [ ]:
np.unique(out)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
plt.imshow(out[0].T, cmap='gray', origin='lower')
plt.show()

In [ ]:
plt.imshow(y_test[0].T, cmap='gray', origin='lower')

In [ ]:
# https://github.com/aurora95/Keras-FCN/blob/master/utils/BilinearUpSampling.py

import keras.backend as K
from keras.layers import Layer, InputSpec
import tensorflow as tf

# How is this different from keras.layers.UpSampling2D?

def resize_images_bilinear(X, height_factor=1, width_factor=1, target_height=None, target_width=None, data_format='default'):
    '''Resizes the images contained in a 4D tensor of shape
    - [batch, channels, height, width] (for 'channels_first' data_format)
    - [batch, height, width, channels] (for 'channels_last' data_format)
    by a factor of (height_factor, width_factor). Both factors should be
    positive integers.
    '''
    if data_format == 'default':
        data_format = K.image_data_format()
    if data_format == 'channels_first':
        original_shape = K.int_shape(X)
        if target_height and target_width:
            new_shape = tf.constant(np.array((target_height, target_width)).astype('int32'))
        else:
            new_shape = tf.shape(X)[2:]
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('int32'))
        X = permute_dimensions(X, [0, 2, 3, 1])
        X = tf.image.resize_bilinear(X, new_shape)
        X = permute_dimensions(X, [0, 3, 1, 2])
        if target_height and target_width:
            X.set_shape((None, None, target_height, target_width))
        else:
            X.set_shape((None, None, original_shape[2] * height_factor, original_shape[3] * width_factor))
        return X
    elif data_format == 'channels_last':
        original_shape = K.int_shape(X)
        if target_height and target_width:
            new_shape = tf.constant(np.array((target_height, target_width)).astype('int32'))
        else:
            new_shape = tf.shape(X)[1:3]
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('int32'))
        X = tf.image.resize_bilinear(X, new_shape)
        if target_height and target_width:
            X.set_shape((None, target_height, target_width, None))
        else:
            X.set_shape((None, original_shape[1] * height_factor, original_shape[2] * width_factor, None))
        return X
    else:
        raise Exception('Invalid data_format: ' + data_format)

class BilinearUpSampling2D(Layer):
    def __init__(self, size=(1, 1), target_size=None, data_format='default', **kwargs):
        if data_format == 'default':
            data_format = K.image_data_format()
        self.size = tuple(size)
        if target_size is not None:
            self.target_size = tuple(target_size)
        else:
            self.target_size = None
        assert data_format in {'channels_last', 'channels_first'}, 'data_format must be in {tf, th}'
        self.data_format = data_format
        self.input_spec = [InputSpec(ndim=4)]
        super(BilinearUpSampling2D, self).__init__(**kwargs)

    def get_output_shape_for(self, input_shape):
        if self.data_format == 'channels_first':
            width = int(self.size[0] * input_shape[2] if input_shape[2] is not None else None)
            height = int(self.size[1] * input_shape[3] if input_shape[3] is not None else None)
            if self.target_size is not None:
                width = self.target_size[0]
                height = self.target_size[1]
            return (input_shape[0],
                    input_shape[1],
                    width,
                    height)
        elif self.data_format == 'channels_last':
            width = int(self.size[0] * input_shape[1] if input_shape[1] is not None else None)
            height = int(self.size[1] * input_shape[2] if input_shape[2] is not None else None)
            if self.target_size is not None:
                width = self.target_size[0]
                height = self.target_size[1]
            return (input_shape[0],
                    width,
                    height,
                    input_shape[3])
        else:
            raise Exception('Invalid data_format: ' + self.data_format)

    def call(self, x, mask=None):
        if self.target_size is not None:
            return resize_images_bilinear(x, target_height=self.target_size[0], target_width=self.target_size[1], data_format=self.data_format)
        else:
            return resize_images_bilinear(x, height_factor=self.size[0], width_factor=self.size[1], data_format=self.data_format)

    def get_config(self):
        config = {'size': self.size, 'target_size': self.target_size}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [ ]:
# https://github.com/fchollet/keras/issues/4465#issuecomment-262229784
# https://github.com/aurora95/Keras-FCN/blob/master/models.py#L23
# https://github.com/aurora95/Keras-FCN/blob/master/models.py#L82

n_classes = 2
weight_decay = 0.

# Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False,
                         input_shape=(224, 224, 3))
model_vgg16_conv.trainable = False

slice_input = Input(shape=(224, 224, 3), name='slice_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(slice_input)

x = Dropout(0.5)(output_vgg16_conv)

# Classifying layer.
x = Convolution2D(n_classes, (1, 1), kernel_initializer='he_normal', 
                  activation='linear', padding='valid', strides=(1, 1), 
                  kernel_regularizer=l2(weight_decay))(x)

# x = UpSampling2D(size=(32, 32))(x)
x = BilinearUpSampling2D(size=(32, 32))(x)

model = Model(slice_input, x)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adadelta', loss='binary_crossentropy',
              metrics=['binary_crossentropy'])

In [ ]:
predict_slice = 50

out = model.predict(X_train[predict_slice])
print("Original:", out.shape)
out = out.squeeze()
print("Reshaped:", out.shape)

fig, ax = plt.subplots(1, 1)

img_labels = np.argmax(out, axis=2)

ax.imshow(X_train[predict_slice].squeeze()[:, :, 1].T, cmap='gray')
ax.imshow(np.ma.masked_where(img_labels<0.5, img_labels), alpha=.7,
          cmap='autumn', interpolation='none')

In [ ]:
model.fit(X_train, y_train, batch_size=20)

In [ ]:
import h5py

with h5py.File('/data/metasearch-anatomical-brainmask-slices.h5', 'r') as fp:
    X_train = fp['/anatomical/axial'][:100]
    y_train = fp['/anatomical/axial'][:100]

In [ ]:
def generate_arrays_from_hdf5(path, start=0, end=None, view='axial', normalizer=None):
    import h5py

    X_dataset = "/anatomical/{}".format(view)
    y_dataset = "/brainmask/{}".format(view)

    with h5py.File(path, 'r') as fp:
        for i in range(start, end):
            X = fp[X_dataset][i]
            y = fp[y_dataset][i]
            
            if normalizer is not None:
                X = normalizer(X)
                y = normalizer(y)
            
            yield (X, y)

data_path = '/data/metasearch-anatomical-brainmask-slices.h5'

generator = generate_arrays_from_hdf5(data_path, end=100, normalizer=process_slice)

model.fit_generator(generator, steps_per_epoch=10, epochs=2)

In [ ]:

weight_decay = 0.

model_vgg16_conv = VGG16(include_top=True, weights='imagenet')
# model_vgg16_conv.trainable = False

input_ = Input(shape=(256, 256, 3), name='slice_input')
output_vgg16_conv = model_vgg16_conv(input_)

x = Flatten(name='flatten')(output_vgg16_conv)
# x = Convolution2D(4096, 7, 7, activation='relu', border_mode='same', name='fc1', W_regularizer=l2(weight_decay))(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
# Classifying layer.
x = Convolution2D(2, 1, 1, init='he_normal', activation='linear', border_mode='valid', subsample=(1, 1), W_regularizer=l2(weight_decay))(x)
x = BilinearUpSampling2D(size=(32, 32))(x)
model = Model(input_, x)

In [ ]:
# https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069



In [ ]:
model = VGG16(include_top=True, weights='imagenet')
model.trainable = False
print(model.summary())

In [ ]:
inp = Input(shape=(1,224,224))

In [ ]:
x = model(inp)

In [ ]:
out = model.predict(X_train[10])
print(np.argmax(out))

In [ ]:
model.summary()

In [ ]:
# one_slice = Input(shape=(None, 256, 256, 3))
cnn = VGG16(include_top=True, weights='imagenet')
cnn.trainable = False

In [ ]:
x = Dense(128, activation='relu')(cnn)
outputs = Dense(1000)(x)